In [7]:
import glob
from jwst.associations.lib.member import Member
from jwst.associations.asn_from_list import asn_from_list
from jwst.associations.lib.rules_level2_base import DMSLevel2bBase

my_science_files = glob.glob('IMA_science/*rate.fits')
my_background_files = glob.glob('IMA_background/*rate.fits')

asn = asn_from_list(my_science_files, rule=DMSLevel2bBase)
asn.data['products'] = None

for n, sci in enumerate(my_science_files):
    
    asn.new_product('my_galaxy_dither{}'.format(str(n+1)))
    sci_member = Member({'expname': sci, 'exptype': 'science'})    
    new_members = asn.current_product['members']
    new_members.append(sci_member)
    
    for bkg in my_background_files:
        bkg_member = Member({'expname': bkg, 'exptype': 'background'})
        new_members.append(bkg_member)

name, ser = asn.dump() 
#print(ser)
print(asn.data['asn_name'])


KeyError: 'asn_name'

In [2]:
my_science_files = glob.glob('IMA_science/*rate.fits')
my_background_files = glob.glob('IMA_background/*rate.fits')

In [86]:
from jwst.associations.asn_from_list import asn_from_list
from jwst.associations.lib.rules_level2_base import DMSLevel2bBase

level2_asn = asn_from_list(my_science_files, rule=DMSLevel2bBase)
name, serialized = level2_asn.dump()

with open('level2_asn.json', 'w') as f:
    f.write(serialized)

In [88]:
print(serialized)

{
    "asn_type": "None",
    "asn_rule": "DMSLevel2bBase",
    "version_id": null,
    "code_version": "0.13.7",
    "degraded_status": "No known degraded exposures in association.",
    "program": "noprogram",
    "products": [
        {
            "name": "IMA_science/det_image_seq2_MIRIMAGE_F1130Wexp1",
            "members": [
                {
                    "expname": "IMA_science/det_image_seq2_MIRIMAGE_F1130Wexp1_rate.fits",
                    "exptype": "science"
                }
            ]
        },
        {
            "name": "IMA_science/det_image_seq1_MIRIMAGE_F1130Wexp1",
            "members": [
                {
                    "expname": "IMA_science/det_image_seq1_MIRIMAGE_F1130Wexp1_rate.fits",
                    "exptype": "science"
                }
            ]
        }
    ]
}


In [98]:
print(type(level2_asn))
print(level2_asn['asn_type'])

level2_asn['asn_type'] = 'ss'

print(level2_asn['asn_type'])

level2_asn._add_items(my_background_files)
print(level2_asn)

print(serialized)

<class 'jwst.associations.lib.rules_level2_base.DMSLevel2bBase'>
ss
ss
Association jwnoprogram-o999_ss_019_asn
    Parameters:        Product type: indetermined        Rule:         indetermined        Program:      indetermined        Target:       indetermined        Pool:         indetermined
        
	IMA_science/det_image_seq2_MIRIMAGE_F1130Wexp1 with 1 members
	IMA_science/det_image_seq1_MIRIMAGE_F1130Wexp1 with 1 members
	IMA_background/det_image_seq1_MIRIMAGE_F1130Wexp1 with 1 members
	IMA_background/det_image_seq1_MIRIMAGE_F1130Wexp1 with 1 members
	IMA_background/det_image_seq1_MIRIMAGE_F1130Wexp1 with 1 members


{
    "asn_type": "None",
    "asn_rule": "DMSLevel2bBase",
    "version_id": null,
    "code_version": "0.13.7",
    "degraded_status": "No known degraded exposures in association.",
    "program": "noprogram",
    "products": [
        {
            "name": "IMA_science/det_image_seq2_MIRIMAGE_F1130Wexp1",
            "members": [
                {
               

In [104]:
for product in level2_asn['products']:
    members = product['members']
    
    for member in members:
        #back = Member()
        #member.append(my_background_files)
        print(member)

{'expname': 'IMA_science/det_image_seq2_MIRIMAGE_F1130Wexp1_rate.fits', 'exptype': 'science'}
{'expname': 'IMA_science/det_image_seq1_MIRIMAGE_F1130Wexp1_rate.fits', 'exptype': 'science'}
{'expname': 'IMA_background/det_image_seq1_MIRIMAGE_F1130Wexp1.fits', 'exptype': 'science'}
{'expname': 'IMA_background/det_image_seq1_MIRIMAGE_F1130Wexp1.fits', 'exptype': 'science'}
{'expname': 'IMA_background/det_image_seq1_MIRIMAGE_F1130Wexp1.fits', 'exptype': 'science'}


In [ ]:
for product in self['products']:
    members = product['members']

    # Split out the science vs. non-science
    # The non-science exposures will get attached
    # to every resulting association.
    science_exps = [
        member
        for member in members
        if member['exptype'] == 'science'
    ]
    nonscience_exps = [
        member
        for member in members
        if member['exptype'] != 'science'
]
    
    
# Create new associations for each science, using
# the other science as background.
results = []
for science_exp in science_exps:
    asn = copy.deepcopy(self)
    asn.data['products'] = None

    product_name = remove_suffix(
        splitext(split(science_exp['expname'])[1])[0]
    )[0]
    asn.new_product(product_name)
    new_members = asn.current_product['members']
    new_members.append(science_exp)

    for other_science in science_exps:
        if other_science['expname'] != science_exp['expname']:
            now_background = Member(other_science)
            now_background['exptype'] = 'background'
            new_members.append(now_background)

    new_members += nonscience_exps

    if asn.is_valid:
        results.append(asn)

return results